# 이데일리 전처리 -> Ngram, Label 정리
**목표 테이블**
- 날짜	매체	제목	내용	N-gram
- date	press	title	content	ng

**순서**
- csv 불러오기
- 데이터프레임 붙이기 - 위,아래로
- date -> date type으로 변경

- 이데일리만 뽑기 -isin or contain?
- string 전처리- \n,\t 등 없앨 것들, 기자 이름, 저작권 표시, 광고 문구 등 제거


In [4]:
import pandas as pd
import numpy as np
import re

## 크롤링한 데이터 전처리 시작

In [200]:
path = 'edaily(05_17).csv'
df = pd.read_csv(path, index_col=0 ,engine='python', header=0) # index_col=0 -> Unnamed 0 안나오게
df.info()
df.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 84283 entries, 0 to 85606
Data columns (total 3 columns):
content    84213 non-null object
date       84259 non-null object
title      84245 non-null object
dtypes: object(3)
memory usage: 2.6+ MB


,content,date,title
0,지난해 11월까지 외화차입가산금리가 하향 안정세를 지속한 것으로 나타났다. 국제금...,2005-01-02,은행 외화차입여건 개선..가산금리 하향안정
1,(단위=% %P)국고채(3년)3.35-0.01회사채(3년)(AA-)3.83보합통안증...,2005-01-10,(표)10일 오전 채권시장 - 금리동향


In [190]:
df['date'].value_counts()
# date에 content가 들어간 것들이 약간 있다.

2016-12-15                                 143
2015-12-17                                 138
2011-01-13                                 114
2008-10-09                                 109
2017-03-16                                 101
2011-02-11                                  97
2010-07-09                                  96
2015-03-12                                  89
2011-05-13                                  88
2010-10-20                                  88
2011-03-10                                  85
2008-10-27                                  84
2008-12-17                                  84
2008-08-07                                  83
2010-11-16                                  82
2011-02-09                                  81
2015-12-16                                  80
2011-06-10                                  79
2007-09-19                                  76
2011-04-12                                  74
2017-06-15                                  74
2015-03-19   

In [197]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84283 entries, 0 to 85606
Data columns (total 3 columns):
content    84213 non-null object
date       84283 non-null object
title      84245 non-null object
dtypes: object(3)
memory usage: 2.6+ MB


In [198]:
df.isnull().count()

content    84283
date       84283
title      84283
dtype: int64

In [ ]:
# 일단 df backup
df_original = df.copy()

#### 전체 데이터에서 1000개 랜덤 샘플 뽑아서 전처리 후, 전체 데이터에 적용했음.
- 이데일리 총 데이터 약 84000개이므로 전처리 시간이 오래 걸릴 것을 고려
- 샘플 1000개만 뽑아서 전처리 진행.
- 샘플이 잘 처리되는 것 확인 후, 전체 데이터로 진행

In [81]:
# # 샘플링
# df_sample =df.sample(n=1000, random_state=1)
# tmp = df_sample.copy()
# tmp.head(3)


##### 샘플링 진행 후, 전체 데이터로 다시 진행
tmp = df.copy()

,content,date,title,url
19870,"\n\t,\n\t,\n\n,\n,\n\t\n\t[edaily 하정민기자] 60달러를...",2005-06-28,"""고유가, 한국 등 아시아 경제 위협""-FT",https://news.naver.com/main/read.nhn?mode=LSD&...
10858,"\n\t,\n\t,\n\n,\n,\n\t\n\t[edaily 이학선기자] 채권금리가...",2005-03-31,채권금리 약보합..`한박자 쉬고‥`(마감),https://news.naver.com/main/read.nhn?mode=LSD&...
22952,"\n\t,\n\t,\n\n,\n,\n\t\n\t[뉴욕=이데일리 안근모특파원] 27일...",2005-07-28,달러화 약세..기술적 저항,https://news.naver.com/main/read.nhn?mode=LSD&...
25809,"\n\t,\n\t,\n\n,\n,\n\t\n\t[이데일리 공희정기자] 지난주 주식형...",2005-08-21,사상최고점서 `삐끗`..주식형 줄줄이 마이너스,https://news.naver.com/main/read.nhn?mode=LSD&...
24728,"\n\t,\n\t,\n\n,\n,\n\t\n\t[이데일리 강종구기자] 하반기에 우리...",2005-08-11,"박승총재 ""성장률 전망 달성 가능""(6보)",https://news.naver.com/main/read.nhn?mode=LSD&...


In [167]:
# url은 필요 없음.
tmp = tmp[['content','date','title']]

## 이데일리 텍스트 정제 by Regural expression

### 텍스트 전처리 : content

In [ ]:
# content(기사 원문)에서 지워야할 부분 예시
'''
앞에 -> 
[이데일리 이유미 기자]
뒤에 ->
이유미 (miyah31@edaily.co.kr)
[종합 경제 뉴스 이데일리 모바일 웹][이데일리 TV][이데일리 ON][MP트래블러][마켓포인트][투자정보 앱 투자플러스][이데일리 창업]
＜ⓒ종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지＞
'''

In [0]:
ex = '''
[뉴욕=edaily 정명수특파원] 
전날 플래시메모리 가격 하락으로 4분기 순손실을 봤다고 밝힌 AMD는 보합선에 머물렀다.
Copyrightⓒ 2000-2005 edaily. All rights reserved.
정명수 뉴욕특파원 (yjs@edaily.co.kr)
[edaily 윤진섭기자]주택구입과 전세자금, 그리고 건설자금 금리가 0.5%포인트 인하된다. 
윤진섭 기자 (yjs@edaily.co.kr)'''

In [171]:
## 기사 앞 뒤에 필요없는 문장 제거

# re.sub(pattern, repl, string)	string에서 pattern과 매치하는 텍스트를 repl로 치환한다
## df = df.replace(r'\\n',' ', regex=True)
## ^\[.+ :  ^:~로 시작하는.

tmp['content'] = tmp['content'].replace(['\n','\t', ','], '', regex=True)
tmp['content'] = tmp['content'].apply(lambda x: re.sub(".+[기자\]]{3}|.+[특파원\]]{4}|.+[증권부\]]{4}|.+[PD\]]{3}",'',x,1))
tmp['content'] = tmp['content'].apply(lambda x: re.sub(".+[edaily\]]{7}|.+[이데일리\]]{5}",'',x,1))
tmp['content'] = tmp['content'].apply(lambda x: re.sub("[종합]{2}.+|[Copyright]{9}.+|\w*\@.+",'',x,1))
tmp['content'] = tmp['content'].apply(lambda x: re.sub("(\\r)",'',x,1))
tmp.head(10)

,content,date,title
0,지난해 11월까지 외화차입가산금리가 하향 안정세를 지속한 것으로 나타났다. 국제금...,2005-01-02,은행 외화차입여건 개선..가산금리 하향안정
1,(단위=% %P)국고채(3년)3.35-0.01회사채(3년)(AA-)3.83보합통안증...,2005-01-10,(표)10일 오전 채권시장 - 금리동향
2,지표금리가 10일 중장기물 약세에 따른 반사이익을 얻으며 강보합권을 걷고 있다. ...,2005-01-10,"채권금리, 종목별 차별화..3년금리↓ 5년금리↑(오전)"
3,채권금리가 연 이틀 급등하며 장기채권의 경우 지난해 8월 콜금리 인하 이전 수준으...,2005-01-14,`무너진 심리`..채권금리 이틀째 폭등(마감)
4,(단위=% %P)국고채(3년)3.70+0.02회사채(3년)(AA-)4.16+0.0...,2005-01-20,(표)20일 오전 채권시장 - 금리동향
5,채권수익률이 20일 매도심리가 확산되며 상승 마감했다. 전날 한국은행의 채권 단순...,2005-01-20,`나 떨고 있니?`..채권금리 상승
6,미국 부자들이 올해 주식 투자비중을 늘리고 채권 투자비중을 줄일 계획이라고 CBS...,2005-01-10,"미국 부자들 ""올해는 채권보다 주식"""
7,채권수익률이 19일 전날 급락에 따른 경계감이 작용하며 약보합권을 걷고 있다. 수...,2005-01-19,"채권금리, 약보합..한은 채권매입에 촉각(오전)"
8,채권시장은 물량부담에 대한 불안감이 투자심리를 위축시켰고 주식시장에서는 단기 급등...,2005-01-20,(마켓서핑)싹트는 불안감
9,중소기업청은 경남은행과 중소기업의 자금난을 완화하기 위한 `중소기업 금융지원 협약...,2005-01-10,"중기청·경남은행, 매년 5천억 中企 지원"


In [224]:
# 아직도 나오는 것들이 있지만, 소수여서 그냥 진행
tmp['content'].value_counts()
'''
[이데일리 증권시장부]
 [이데일리 시장부]   
 [이데일리 채권외환팀] 
 이데일리ⓒ 1등 경제정보 멀티미디어.증권부
 [이데일리N] 
 [이데일리 투자전략팀]
'''

In [172]:
# 예시로 하나 원문 보기
tmp['content'][19870]

' 경기 후퇴를 우려하는 미국이 금리 인하를 단행 달러 약세가 점쳐지는 가운데 글로벌 대형 금융기관들이 엔화로 발행되는 `사무라이본드`\xa0발행을 늘리고 있다.\xa0 호주 4위 은행인 웨스트팩은 지난 23일\xa0처음으로 일본 금융시장에서 사무라이본드를 선보였다고 일본 니혼게이자이신문이 다우존스를 인용해 보도했다. 270억엔 규모의 5년만기 사무라이본드를 1.54% 고정금리로\xa0발행하고 500억엔 규모의 5년만기 변동금리 사무라이본드를 리보금리에 53bp를 가산해 발행했다. 세계 최대은행인 골드만삭스도 183억엔 규모의\xa0사무라이본드를 2.11%의 고정금리에\xa0발행했다. 골드만삭스는 변동금리 사무라이본드의 경우 이보다 10배 많은 1302억엔 규모로 유로엔 리보금리에 1.1%포인트를 가산한 수준에 발행했다. 앞서 지난 18일 모간스탠리도 2년여만에 502억달러 규모의 사무라이본드를 발행할 것으로 예상된다고 블룸버그통신이 보도했다. 서브프라임 모기지 부실로 피해를 입은 미국의 금융기관을 포함해 해외 유수 금융기관들이 사무라이본드를 통한 해외에서의 자금 조달에 눈을 돌리고 있다. 미국 연방준비제도이사회(FRB)의 금리인하 기조와 함께 달러약세가 지속될 것으로 예상되자 달러가 아닌 대체 통화 채권으로\xa0갈아타고 있는 것. 특히 금리가 낮은 일본 시장에서는 조금만 금리를 높여도 투자자들을 끌어모을 수 있다는 분석이다. 또 일본은 상대적으로 미국의 서브프라임 모기지 부실의 악영향이 상대적으로 적어 주목받고 있다. UBS증권(아시아태평양)의 니콜라스 로스 채권시장 수석은 "일본 투자자들의 대부분은 서브프라임 위기에도 투자 이익을 유지해왔기 때문에 웨스트백의 사무라이본드 시장 데뷔는 순조롭게 진행되고 있다"고 말했다. 시장에서는 만기가 중장기적이면서 적용 이자율은 단기 시장이자율을 반영하도록 설계된 변동금리부사채가 각광받고 있다. 변동금리부사채의 발행자는 시장금리가 낮아질 경우 이익이다. 이를 매입하는 투자자들은 금리 상승시 이자소득이 증가하며 대체적으로

In [ ]:
## csv로파일 백업
# save_file = 'edaily_preprocessed.csv'
# tmp.to_csv(save_file, index=False)

## Tokenize, Ngramize

In [5]:
path='edaily_preprocessed.csv'
df = pd.read_csv(path, engine='python', index_col=0, header=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84283 entries, 0 to 84282
Data columns (total 4 columns):
Unnamed: 0    84272 non-null object
content       84213 non-null object
date          84259 non-null object
title         84245 non-null object
dtypes: object(4)
memory usage: 2.6+ MB


In [178]:
# ngramize 전에 string으로 타입 변환
df = df.astype('str')

In [199]:
# 데이터 파악
df.describe()
df.info()

,content,date,title
count,84259,84259,84259
unique,80516,4514,79681
top,\r,2016-12-15,이번주 재테크 캘린더
freq,2036,143,88


In [8]:
# Ngram 전에도 샘플링 진행했음.
# df_sample =df.sample(n=1000, random_state=1)
# tmp = df_sample.copy()
# tmp.head(3)


##### 샘플링 진행 후, 전체 데이터로 다시 진행
df_original = df.copy()

In [11]:
df = df_original.copy()

### Ngramize
- ekonlpy의 MPCK 사용, tokens, ngram 생성
- 불용어 처리 포함

In [9]:
# tokenize, ngramize 함수 정의 from ekonlpy
import pandas as pd
from ekonlpy.sentiment import MPCK

def text2ngram(text) :
    mpck = MPCK()
    bef_tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(bef_tokens)
    tokens = []
    
    # 불용어 처리
    stoppos = ['SC','SY','SF','SE','SS','SP','SO','SW','SSC','JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC','EF','EC','ETN','ETM']
    stopword = ['']

    for tag in bef_tokens:
        if tag[1] not in stoppos:
            if tag[0] not in stopword:
                tokens.append(tag)
                
    return tokens, ngrams

In [12]:
# 토큰, ngram 생성
#########  매우 오래 걸림: 약 5시간(이데일리 84000개)  ########
import re 

df['tokens'] = ''
df['ngram'] = ''

for i, row in df.iterrows() :
    try :
        text = row['content']
        tokens,ngrams = text2ngram(text)

        row['tokens'] = ','.join(tokens) # row에 .join으로 붙여줌
        row['ngram'] = ','.join(ngrams)
        
        if i%1000 == 0 : # %: 나머지, 1000개마다 프린트
            print('{}/{}'.format(i, df.shape[0]))    
    
    except Exception as ex: # 에러나면 에러 이름
        print("============== {} : {} ==============".format(i, str(ex)))
#         print(row['content'])

# save_file = 'ngramized.csv'
# df.to_csv(save_file, index=False)
df.head()

0/100


,content,date,title,tokens,ngram
0,지난해 11월까지 외화차입가산금리가 하향 안정세를 지속한 것으로 나타났다. 국제금...,2005-01-02,은행 외화차입여건 개선..가산금리 하향안정,"외화차입/NNG,스프레드/NNG,하향/NNG,안정/NNG,지속/NNG,나타나/VV,...","거주자/NNG;외화예금/NNG;감소/NNG,거주자/NNG;외화예금/NNG;증가/NN..."
1,(단위=% %P)국고채(3년)3.35-0.01회사채(3년)(AA-)3.83보합통안증...,2005-01-10,(표)10일 오전 채권시장 - 금리동향,"단위/NNG,국고채/NNG,회사채/NNG,보합/NNG,통안채/NNG,통안채/NNG,...",
2,지표금리가 10일 중장기물 약세에 따른 반사이익을 얻으며 강보합권을 걷고 있다. ...,2005-01-10,"채권금리, 종목별 차별화..3년금리↓ 5년금리↑(오전)","지표/NNG,금리/NNG,중장기물/NNG,약세/NNG,따르/VV,반사이익/NNG,얻...","국고채/NNG;물량/NNG;부담/NNG,물량/NNG;부담/NNG;경계/NNG,콜/N..."
3,채권금리가 연 이틀 급등하며 장기채권의 경우 지난해 8월 콜금리 인하 이전 수준으...,2005-01-14,`무너진 심리`..채권금리 이틀째 폭등(마감),"채권/NNG,금리/NNG,연/VV,급등/NNG,하/XSV,장기물/NNG,콜/NNG,...","국고채/NNG;오르/VV;국고채/NNG;오르/VV,유료뉴스/NNG;시장/NNG;플러..."
4,(단위=% %P)국고채(3년)3.70+0.02회사채(3년)(AA-)4.16+0.0...,2005-01-20,(표)20일 오전 채권시장 - 금리동향,"단위/NNG,국고채/NNG,회사채/NNG,통안채/NNG,통안채/NNG,cd/NNG,...",


In [13]:
# tokens, ngram 잘 됐나 확인
print(df['tokens'][0])
print(df['ngram'][0])

'거주자/NNG;외화예금/NNG;감소/NNG,거주자/NNG;외화예금/NNG;증가/NNG,외화차입/NNG;여건/NNG;개선/NNG,단기물/NNG;소폭/MAG;상승/NNG,하향/NNG;안정/NNG,금리/NNG;안정/NNG,수출/NNG;호조/NNG,경기/NNG;부진/NNG'

In [ ]:
### 재현시 ngramize 스킵하고 이미 완성된 csv 불러와서 진행
### 재현시 ngramize 스킵하고 이미 완성된 csv 불러와서 진행
### 재현시 ngramize 스킵하고 이미 완성된 csv 불러와서 진행

In [122]:
### 재현시 ngramize 스킵하고 이미 완성된 csv 불러와서 진행

path = 'ngramized.csv'
df = pd.read_csv(path, engine='python', index_col=0, header=0,sep=',')
df.head(3)

,content,date,title,tokens,ngram
0,지난해 11월까지 외화차입가산금리가 하향 안정세를 지속한 것으로 나타났다. 국제금...,2005-01-02,은행 외화차입여건 개선..가산금리 하향안정,"외화차입/NNG,스프레드/NNG,하향/NNG,안정/NNG,지속/NNG,나타나/VV,...","거주자/NNG;외화예금/NNG;감소/NNG,거주자/NNG;외화예금/NNG;증가/NN..."
1,(단위=% %P)국고채(3년)3.35-0.01회사채(3년)(AA-)3.83보합통안증...,2005-01-10,(표)10일 오전 채권시장 - 금리동향,"단위/NNG,국고채/NNG,회사채/NNG,보합/NNG,통안채/NNG,통안채/NNG,...",NaN
2,지표금리가 10일 중장기물 약세에 따른 반사이익을 얻으며 강보합권을 걷고 있다. ...,2005-01-10,"채권금리, 종목별 차별화..3년금리↓ 5년금리↑(오전)","지표/NNG,금리/NNG,중장기물/NNG,약세/NNG,따르/VV,반사이익/NNG,얻...","국고채/NNG;물량/NNG;부담/NNG,물량/NNG;부담/NNG;경계/NNG,콜/N..."


In [125]:
df = df.astype('str')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84283 entries, 0 to 84282
Data columns (total 5 columns):
content    84283 non-null object
date       84283 non-null object
title      84283 non-null object
tokens     84283 non-null object
ngram      84283 non-null object
dtypes: object(5)
memory usage: 3.2+ MB


In [126]:
# tokens와 ngram을 column 하나로 합친다. = tokgram

df['tokgram'] = df[['tokens', 'ngram']].apply(lambda x: ','.join(x), axis=1)
df.head(1)

,content,date,title,tokens,ngram,tokgram
0,지난해 11월까지 외화차입가산금리가 하향 안정세를 지속한 것으로 나타났다. 국제금...,2005-01-02,은행 외화차입여건 개선..가산금리 하향안정,"외화차입/NNG,스프레드/NNG,하향/NNG,안정/NNG,지속/NNG,나타나/VV,...","거주자/NNG;외화예금/NNG;감소/NNG,거주자/NNG;외화예금/NNG;증가/NN...","외화차입/NNG,스프레드/NNG,하향/NNG,안정/NNG,지속/NNG,나타나/VV,..."
1,(단위=% %P)국고채(3년)3.35-0.01회사채(3년)(AA-)3.83보합통안증...,2005-01-10,(표)10일 오전 채권시장 - 금리동향,"단위/NNG,국고채/NNG,회사채/NNG,보합/NNG,통안채/NNG,통안채/NNG,...",nan,"단위/NNG,국고채/NNG,회사채/NNG,보합/NNG,통안채/NNG,통안채/NNG,..."
2,지표금리가 10일 중장기물 약세에 따른 반사이익을 얻으며 강보합권을 걷고 있다. ...,2005-01-10,"채권금리, 종목별 차별화..3년금리↓ 5년금리↑(오전)","지표/NNG,금리/NNG,중장기물/NNG,약세/NNG,따르/VV,반사이익/NNG,얻...","국고채/NNG;물량/NNG;부담/NNG,물량/NNG;부담/NNG;경계/NNG,콜/N...","지표/NNG,금리/NNG,중장기물/NNG,약세/NNG,따르/VV,반사이익/NNG,얻..."


In [127]:
df_original = df.copy()

In [133]:
# 칼럼 정리
df = pd.DataFrame(df,columns= ['date','press','title','content','tokgram'])

# press에 값 신문사 이름 지정
df['press'] = 'edaily'

In [135]:
# 아까 tokgram으로 이름 정했던거 다시 ngram으로 rename

# df.rename(columns={"tokgram": "ngram"}
df = df.rename(columns={"tokgram": "ngram"})
df.head(3)

,date,press,title,content,ngram
0,2005-01-02,edaily,은행 외화차입여건 개선..가산금리 하향안정,지난해 11월까지 외화차입가산금리가 하향 안정세를 지속한 것으로 나타났다. 국제금...,"외화차입/NNG,스프레드/NNG,하향/NNG,안정/NNG,지속/NNG,나타나/VV,..."
1,2005-01-10,edaily,(표)10일 오전 채권시장 - 금리동향,(단위=% %P)국고채(3년)3.35-0.01회사채(3년)(AA-)3.83보합통안증...,"단위/NNG,국고채/NNG,회사채/NNG,보합/NNG,통안채/NNG,통안채/NNG,..."
2,2005-01-10,edaily,"채권금리, 종목별 차별화..3년금리↓ 5년금리↑(오전)",지표금리가 10일 중장기물 약세에 따른 반사이익을 얻으며 강보합권을 걷고 있다. ...,"지표/NNG,금리/NNG,중장기물/NNG,약세/NNG,따르/VV,반사이익/NNG,얻..."


In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84283 entries, 0 to 84282
Data columns (total 5 columns):
date       84283 non-null object
press      84283 non-null object
title      84283 non-null object
content    84283 non-null object
ngram      84283 non-null object
dtypes: object(5)
memory usage: 3.2+ MB


## 콜금리랑 합치기
- 목표 데이터 프레임: date/press/title/content/ngram/label
- -> date / ngram / label(updown)

In [137]:
# 콜금리 csv 파일 읽어오기
call = pd.read_csv('my_callrate_ffill.csv',engine='python',index_col=0, sep=',',header=0)

In [139]:
call = call.astype('str')
call.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5309 entries, 0 to 5308
Data columns (total 5 columns):
date            5309 non-null object
old_date        5309 non-null object
callrate        5309 non-null object
old_callrate    5309 non-null object
updown          5309 non-null object
dtypes: object(5)
memory usage: 207.5+ KB


In [148]:
## left조인이기 때문에 callrate가 2019년까지 있는 것은 상관 없이 '이데일리' 기준으로 맞춰짐(2005~2017년까지)
upgram = pd.merge(df, call, on='date', how='left')

In [150]:
# 필요한 칼럼만 데이터프레임에 저장
upgram = upgram[['date','ngram','updown']]
upgram.tail(3)

,date,ngram,updown
0,2005-01-02,"외화차입/NNG,스프레드/NNG,하향/NNG,안정/NNG,지속/NNG,나타나/VV,...",up
1,2005-01-10,"단위/NNG,국고채/NNG,회사채/NNG,보합/NNG,통안채/NNG,통안채/NNG,...",down
2,2005-01-10,"지표/NNG,금리/NNG,중장기물/NNG,약세/NNG,따르/VV,반사이익/NNG,얻...",down


In [177]:
upgram.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84283 entries, 0 to 84282
Data columns (total 3 columns):
date      84283 non-null object
ngram     84283 non-null object
updown    84245 non-null object
dtypes: object(3)
memory usage: 2.6+ MB


In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84283 entries, 0 to 84282
Data columns (total 5 columns):
date       84283 non-null object
press      84283 non-null object
title      84283 non-null object
content    84283 non-null object
ngram      84283 non-null object
dtypes: object(5)
memory usage: 3.2+ MB


In [182]:
# # 최종 csv파일 저장 : date, ngram, updown label
# save_file = 'upgram.csv'
# df.to_csv(save_file, index=False)

## ngram 카운트

- value_counts 이용, all, up, down 붙여서 데이터프레임 만들기
- 만들어진 df에서 ngram 수 15개 미만은 버림

In [5]:
path= 'edaily_final_ngram_label.csv'
df = pd.read_csv(path, engine='python',index_col=0, sep=',',header=0)
df.head(3)

,ngram,updown
date,,
2005-01-02,"외화차입/NNG,스프레드/NNG,하향/NNG,안정/NNG,지속/NNG,나타나/VV,...",up
2005-01-10,"단위/NNG,국고채/NNG,회사채/NNG,보합/NNG,통안채/NNG,통안채/NNG,...",down
2005-01-10,"지표/NNG,금리/NNG,중장기물/NNG,약세/NNG,따르/VV,반사이익/NNG,얻...",down


In [208]:
# up, down 분포 보기
df['updown'].value_counts()

down          35454
up            34179
neutrality    14612
Name: updown, dtype: int64

In [6]:
## df_all 만들기: 전체 ngram 카운팅
### 실행 시간 10분 정도 걸림

temp_ls = [] # 빈리스트 생성
label_ls = []

for i in range(len(df)): # 사용할 데이터프레임의 길이로 for문 돌리기
    try:
        test = df['ngram'][i].split(',') # 컬럼이름 ngram의 i 번째를 split
        label = df['updown'][i]
        temp_ls.extend(test) # 빈 리스트에 한번에 추가
        label_ls.extend([label]*len(test))
        
        if i%10000 == 0 : # %: 나머지, 10000개마다 프린트
            print('{}/{}'.format(i, df.shape[0]))

    except Exception as ex: # 에러나면 에러 이름
        print("============== {} : {} ==============".format(i, str(ex)))
#       print(row[‘content’])
        pass

temp_df = pd.DataFrame({'ngram':temp_ls,'updown':label_ls}, columns=['ngram', 'updown']) # 새로운 데이터프레임 생성

## 전체, up, down 데이터 프레임 생성
df_all = pd.DataFrame(temp_df['ngram'].value_counts()) # value_counts사용
df_up = pd.DataFrame(temp_df[temp_df.updown=='up']['ngram'].value_counts())
df_down = pd.DataFrame(temp_df[temp_df.updown=='down']['ngram'].value_counts())

0/84283
10000/84283
20000/84283
30000/84283
40000/84283
50000/84283
60000/84283
70000/84283
80000/84283


In [7]:
df_all.info()
df_up.info()
df_down.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107269 entries, 하/VV to 불기둥/NNG
Data columns (total 1 columns):
ngram    107269 non-null int64
dtypes: int64(1)
memory usage: 1.6+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 81333 entries, 하/VV to 하락/NNG;인플레이션/NNG;억제/NNG
Data columns (total 1 columns):
ngram    81333 non-null int64
dtypes: int64(1)
memory usage: 1.2+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 81695 entries, 하/VV to 원화채권/NNG;보유/NNG;잔고/NNG;줄/VV
Data columns (total 1 columns):
ngram    81695 non-null int64
dtypes: int64(1)
memory usage: 1.2+ MB


In [8]:
df_all.head()

,ngram
하/VV,661988
하/XSV,367142
되/VV,273990
금리/NNG,248548
보/VV,142505


In [9]:
'''reset_index 명령으로 인덱스를 보통의 자료열로 바꿀 수도 있다.
이 때 인덱스 열은 자료열의 가장 선두로 삽입된다.
데이터프레임의 인덱스는 정수로 된 디폴트 인덱스로 바뀐다.'''

'reset_index 명령으로 인덱스를 보통의 자료열로 바꿀 수도 있다.\n이 때 인덱스 열은 자료열의 가장 선두로 삽입된다.\n데이터프레임의 인덱스는 정수로 된 디폴트 인덱스로 바뀐다.'

In [10]:
df_all = df_all.reset_index()
df_up = df_up.reset_index()
df_down = df_down.reset_index()

In [11]:
df_all.head(1)

,index,ngram
0,하/VV,661988


In [12]:
## merge 전에 데이터 백업
df_all_ori = df_all.copy()
df_up_ori = df_up.copy()
df_down_ori = df_down.copy()

In [13]:
### df_full, df_up1, df_down1 조인 on index, how = left
merged = df_all.merge(df_up, how='left', on='index')
merged = merged.merge(df_down, how='left', on='index')
merged.info()
merged.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107269 entries, 0 to 107268
Data columns (total 4 columns):
index      107269 non-null object
ngram_x    107269 non-null int64
ngram_y    81333 non-null float64
ngram      81695 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 4.1+ MB


,index,ngram_x,ngram_y,ngram
0,하/VV,661988,271301.0,275870.0
1,하/XSV,367142,148361.0,155332.0
2,되/VV,273990,114116.0,113072.0


In [14]:
# upgram.rename(columns={"tokgram": "ngram"}
merged = merged.rename(columns={"ngram_x": "all","ngram_y":"up","ngram":"down"})
merged.head(3)

,index,all,up,down
0,하/VV,661988,271301.0,275870.0
1,하/XSV,367142,148361.0,155332.0
2,되/VV,273990,114116.0,113072.0


In [15]:
pd.options.display.float_format = '{:,.0f}'.format
merged.head(3)

,index,all,up,down
0,하/VV,661988,"271,301","275,870"
1,하/XSV,367142,"148,361","155,332"
2,되/VV,273990,"114,116","113,072"


In [16]:
merged

,index,all,up,down
0,하/VV,661988,"271,301","275,870"
1,하/XSV,367142,"148,361","155,332"
2,되/VV,273990,"114,116","113,072"
3,금리/NNG,248548,"101,226","105,520"
4,보/VV,142505,"58,537","58,483"
5,상승/NNG,130423,"56,264","51,821"
6,있/VV,123086,"49,981","51,543"
7,되/XSV,121115,"50,879","50,514"
8,시장/NNG,98652,"40,630","41,257"
9,하락/NNG,96587,"40,103","39,390"


In [17]:
# # csv로파일 백업
# save_file = 'merged.csv'
# merged.to_csv(save_file, index=False)

In [19]:
# ngram 수가 15개 미만인 것들 다 지움.
df_15 = merged.where(merged['all']>=15)
df_15 = df_15.dropna()

,index,all,up,down
0,하/VV,"661,988","271,301","275,870"
1,하/XSV,"367,142","148,361","155,332"
2,되/VV,"273,990","114,116","113,072"
3,금리/NNG,"248,548","101,226","105,520"
4,보/VV,"142,505","58,537","58,483"
5,상승/NNG,"130,423","56,264","51,821"
6,있/VV,"123,086","49,981","51,543"
7,되/XSV,"121,115","50,879","50,514"
8,시장/NNG,"98,652","40,630","41,257"
9,하락/NNG,"96,587","40,103","39,390"


In [20]:
df_15.tail()

,index,all,up,down
24221,후광/NNG,15,10,5
24222,서비스업/NNG;상승/NNG,15,10,3
24223,지수/NNG;상승/NNG;막/VV,15,3,9
24224,베이지북/NNG;경기/NNG;둔화/NNG,15,12,3
24225,원자재/NNG;가격/NNG;내리/VV,15,5,9


## 극성
- 극성= (ngram이 up일 때 등장한 횟수/총 up 수) / (ngram이 down일 때 등장한 횟수/ 총 down 수)

In [110]:
# df_15를 이용하여 진행
df = df_15.copy()

In [111]:
# 판다스 소수점 자리 보이는 것 설정
pd.options.display.float_format = '{:,.5f}'.format

In [112]:
# sum_positive(up), sum_negative(negative) = Numpy 배열로 나옴
sum_up = df['up'].sum()
sum_down = df['down'].sum()

In [113]:
# 극성 수식 = (ngram이 up일 때 등장한 횟수/총 up 수) / (ngram이 down일 때 등장한 횟수/ 총 down 수)
df['bunja'] = df['up']/sum_up
df['bunmo'] = df['down']/sum_down
df['polarity'] = df['bunja']/df['bunmo']
df.head(3)

,index,all,up,down,bunja,bunmo,polarity
0,하/VV,"661,988.00000","271,301.00000","275,870.00000",0.04614,0.04575,1.00852
1,하/XSV,"367,142.00000","148,361.00000","155,332.00000",0.02523,0.02576,0.97948
2,되/VV,"273,990.00000","114,116.00000","113,072.00000",0.01941,0.01875,1.03497


In [114]:
# 0.76~1.3까지 날려버리기
## 판다스, 조건문에 일치하는 것만 가져오기
df = df[(df['polarity']>1.3)|(df['polarity']<0.76)]
df.head()

,index,all,up,down,bunja,bunmo,polarity
17,인상/NNG,"72,007.00000","33,918.00000","26,064.00000",0.00577,0.00432,1.33453
45,물가/NNG,"36,011.00000","17,233.00000","13,164.00000",0.00293,0.00218,1.34249
46,인하/NNG,"35,697.00000","10,977.00000","19,294.00000",0.00187,0.00320,0.58344
90,인플레이션/NNG,"23,348.00000","11,334.00000","8,848.00000",0.00193,0.00147,1.31364
230,위안/NNG,"10,514.00000","4,758.00000","3,751.00000",0.00081,0.00062,1.30081


In [115]:
df1 = df.copy()

In [116]:
## 1.3보다 크면 1('hawkish' 매파), 0.76보다 작으면 -1('dovish' 비둘기파))
# 오류 이유 모르겠으나 실행됨.

df1['hawk/dov'] = 0 # hawk/dov 컬럼 생성, 0을 default로 (중립)
df1['hawk/dov'][df1['polarity']>1.3] = 1
df1['hawk/dov'][df1['polarity']<0.76] = -1

/Users/shinbeompyeon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/shinbeompyeon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [117]:
df1.head()

,index,all,up,down,bunja,bunmo,polarity,hawk/dov
17,인상/NNG,"72,007.00000","33,918.00000","26,064.00000",0.00577,0.00432,1.33453,1
45,물가/NNG,"36,011.00000","17,233.00000","13,164.00000",0.00293,0.00218,1.34249,1
46,인하/NNG,"35,697.00000","10,977.00000","19,294.00000",0.00187,0.00320,0.58344,-1
90,인플레이션/NNG,"23,348.00000","11,334.00000","8,848.00000",0.00193,0.00147,1.31364,1
230,위안/NNG,"10,514.00000","4,758.00000","3,751.00000",0.00081,0.00062,1.30081,1


In [118]:
# 위에서 중립인 것들을 없앴으므로 index가 엉킴 -> 리셋해줌
df1 = df1.reset_index(drop=True)
df1.head()

,index,all,up,down,bunja,bunmo,polarity,hawk/dov
0,인상/NNG,"72,007.00000","33,918.00000","26,064.00000",0.00577,0.00432,1.33453,1
1,물가/NNG,"36,011.00000","17,233.00000","13,164.00000",0.00293,0.00218,1.34249,1
2,인하/NNG,"35,697.00000","10,977.00000","19,294.00000",0.00187,0.00320,0.58344,-1
3,인플레이션/NNG,"23,348.00000","11,334.00000","8,848.00000",0.00193,0.00147,1.31364,1
4,위안/NNG,"10,514.00000","4,758.00000","3,751.00000",0.00081,0.00062,1.30081,1


In [119]:
df1['hawk/dov'].value_counts()

-1    6124
 1    5890
Name: hawk/dov, dtype: int64

In [120]:
# 인덱스 정리
# df.rename(columns={"tokgram": "ngram"}
pd.options.display.float_format = '{:,.2f}'.format
df1 = df1.rename(columns={"index": "ngram"})

In [121]:
df1.drop(['bunja', 'bunmo'], axis='columns', inplace=True)
df1.head()

,ngram,all,up,down,polarity,hawk/dov
0,인상/NNG,"72,007.00","33,918.00","26,064.00",1.33,1
1,물가/NNG,"36,011.00","17,233.00","13,164.00",1.34,1
2,인하/NNG,"35,697.00","10,977.00","19,294.00",0.58,-1


In [122]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12014 entries, 0 to 12013
Data columns (total 6 columns):
ngram       12014 non-null object
all         12014 non-null float64
up          12014 non-null float64
down        12014 non-null float64
polarity    12014 non-null float64
hawk/dov    12014 non-null int64
dtypes: float64(4), int64(1), object(1)
memory usage: 563.2+ KB


In [123]:
df1.describe()

,all,up,down,polarity,hawk/dov
count,"12,014.00","12,014.00","12,014.00","12,014.00","12,014.00"
mean,100.45,40.25,42.61,1.33,-0.02
std,882.25,395.17,353.27,1.47,1.00
min,15.00,1.00,1.00,0.03,-1.00
25%,20.00,8.00,8.00,0.57,-1.00
50%,31.00,13.00,13.00,0.75,-1.00
75%,64.00,26.00,27.00,1.68,1.00
max,"72,007.00","33,918.00","26,064.00",46.15,1.00


In [124]:
# null 값 없음
df1.isnull().count()

ngram       12014
all         12014
up          12014
down        12014
polarity    12014
hawk/dov    12014
dtype: int64

In [125]:
# # csv로파일 백업
# save_file = 'df_hawk_dov.csv'
# df1.to_csv(save_file, index=False)